## BERTopic
BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions.

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [ ]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

In [ ]:
#!pip install transformers #in case of a Greek models?

# Data


In [ ]:
import pandas as pd

In [ ]:
#connect to G-Drive
from google.colab import drive
mount='/content/gdrive/'
drive.mount(mount, force_remount=True)

Mounted at /content/gdrive/


In [ ]:
%cd "/content/gdrive/MyDrive/Colab Notebooks/Topic Modelling"

/content/gdrive/MyDrive/Colab Notebooks/Topic Modelling


In [ ]:
df = pd.read_csv("process-steps-full.csv", sep=',',
    usecols=[0,1,2,3,4,5,6],
    encoding="utf8")

df = df.dropna(subset=['step_title']) #drop also empty values (had 2 -> quality issue)

docs = df["step_title"].values.tolist()

In [ ]:
len(docs)

19931

# **Topic Modeling**



## Training

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>20_000 documents). It is advised to turn this off if you want to speed up the model.


### Stop Words for Count Vectorizer

In [ ]:
# get Greek stop_words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
greek_stopwords = stopwords.words('greek')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
new_words = ['της', 'τη', 'του', 'από']

for word in new_words:
  greek_stopwords.append(word)

### Initialize models & Training

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer # Embeddings
from umap import UMAP #Dimensionality reduction
from hdbscan import HDBSCAN #clustering
from sklearn.feature_extraction.text import CountVectorizer # CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from transformers.pipelines import pipeline

As the number of sample increases the set-up of parameters might have to change in order to have good results. So there is a trail & error game


In [ ]:
#embedding_model = pipeline("feature-extraction", model="nlpaueb/bert-base-greek-uncased-v1") #There is also the option of greek model, but after some test the other model worked better
embedding_model = SentenceTransformer("all-mpnet-base-v2")
umap_model = UMAP(n_neighbors=100, n_components=8, min_dist=0.1, metric='cosine') #increasing n_neighbors in general larger clusters | n_components - in which dimension. Too small loss of info, too high performance | min_dist how far the points should be in low dimensional
hdbscan_model = HDBSCAN(min_cluster_size=8, metric='euclidean', cluster_selection_method='eom') #min_cluster_size, increasing -> fewer clusters | cluster_selection_method also the "leaf" method available (smaller clusters)
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=greek_stopwords) #ngram_range combination of words
ctfidf_model = ClassTfidfTransformer(bm25_weighting=True)

In [ ]:
topic_model = BERTopic(embedding_model=embedding_model, umap_model=umap_model, hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model, ctfidf_model=ctfidf_model)
topics, probs = topic_model.fit_transform(docs)

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents.

In [ ]:
freq = topic_model.get_topic_info(); freq.head(20)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,6730,-1_υπηρεσιών_καταγγελίας_αξιολόγησης_τους,"[υπηρεσιών, καταγγελίας, αξιολόγησης, τους, έγ...","[Έλεγχος του φακέλου της αίτησης, Έκδοση Υπουρ..."
1,0,350,0_συνέχεια_πρωτοβάθμιας_λοιπών δικαιολογητικών...,"[συνέχεια, πρωτοβάθμιας, λοιπών δικαιολογητικώ...",[Παραλαβή και έλεγχος της αίτησης και λοιπών δ...
2,1,330,1_εκκίνηση διαδικασίας εκκίνηση_διαδικασίας εκ...,"[εκκίνηση διαδικασίας εκκίνηση, διαδικασίας εκ...","[Εκκίνηση διαδικασίας, Εκκίνηση διαδικασίας, Ε..."
3,2,272,2_έλεγχος δικαιολογητικών έλεγχος_δικαιολογητι...,"[έλεγχος δικαιολογητικών έλεγχος, δικαιολογητι...","[Έλεγχος Δικαιολογητικών, Έλεγχος Δικαιολογητι..."
4,3,203,3_δικαιολογητικών ηλεκτρονική υποβολή_ηλεκτρον...,"[δικαιολογητικών ηλεκτρονική υποβολή, ηλεκτρον...",[Ηλεκτρονική Υποβολή Αίτησης και Δικαιολογητικ...
5,4,179,4_υποβολή αίτησης υποβολή_αίτησης υποβολή αίτη...,"[υποβολή αίτησης υποβολή, αίτησης υποβολή αίτη...","[Υποβολή αίτησης, Υποβολή αίτησης, Υποβολή αίτ..."
6,5,144,5_έκδοση απόφασης έκδοση_απόφασης έκδοση απόφα...,"[έκδοση απόφασης έκδοση, απόφασης έκδοση απόφα...","[Έκδοση Απόφασης, Έκδοση απόφασης, Έκδοση απόφ..."
7,6,141,6_απόρριψη αιτήματος απόρριψη_αιτήματος απόρρι...,"[απόρριψη αιτήματος απόρριψη, αιτήματος απόρρι...","[Απόρριψη αίτησης, Απόρριψη αίτησης, Απόρριψη ..."
8,7,136,7_αίτησης πρωτοκόλληση αίτησης_πρωτοκόλληση αί...,"[αίτησης πρωτοκόλληση αίτησης, πρωτοκόλληση αί...","[Πρωτοκόλληση Αίτησης, Πρωτοκόλληση Αίτησης, Π..."
9,8,132,8_παραλαβή δικαιολογητικών παραλαβή_δικαιολογη...,"[παραλαβή δικαιολογητικών παραλαβή, δικαιολογη...","[Παραλαβή αίτησης και δικαιολογητικών, Παραλαβ..."


In [ ]:
freq["Topic"].nunique()

529

-1 refers to all outliers. But this amount is quite high, more than 30% of out data. So we would like to reduce the noise. Closely related items will be assigned in the same topic. And in that topic they would be recognised as unique in the similarity process

In [ ]:
new_topics = topic_model.reduce_outliers(docs, topics, strategy="embeddings", threshold=0.90)

In [ ]:
topic_model.update_topics(docs, topics=new_topics, vectorizer_model=vectorizer_model,  ctfidf_model=ctfidf_model)

In [ ]:
freq = topic_model.get_topic_info(); freq.head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1976,-1_αναφοράς_σχετικά_1ο_1ο μέρος,"[αναφοράς, σχετικά, 1ο, 1ο μέρος, ως, μέτρων, ...","[Έλεγχος του φακέλου της αίτησης, Έκδοση Υπουρ..."
1,0,350,0_συνέχεια_πρωτοβάθμιας_λοιπών δικαιολογητικών...,"[συνέχεια, πρωτοβάθμιας, λοιπών δικαιολογητικώ...",[Παραλαβή και έλεγχος της αίτησης και λοιπών δ...
2,1,330,1_εκκίνηση διαδικασίας εκκίνηση_διαδικασίας εκ...,"[εκκίνηση διαδικασίας εκκίνηση, διαδικασίας εκ...","[Εκκίνηση διαδικασίας, Εκκίνηση διαδικασίας, Ε..."
3,2,278,2_έλεγχος δικαιολογητικών έλεγχος_δικαιολογητι...,"[έλεγχος δικαιολογητικών έλεγχος, δικαιολογητι...","[Έλεγχος Δικαιολογητικών, Έλεγχος Δικαιολογητι..."
4,3,209,3_δικαιολογητικών ηλεκτρονική υποβολή_ηλεκτρον...,"[δικαιολογητικών ηλεκτρονική υποβολή, ηλεκτρον...",[Ηλεκτρονική Υποβολή Αίτησης και Δικαιολογητικ...
5,4,183,4_υποβολή αίτησης υποβολή_αίτησης υποβολή αίτη...,"[υποβολή αίτησης υποβολή, αίτησης υποβολή αίτη...","[Υποβολή αίτησης, Υποβολή αίτησης, Υποβολή αίτ..."
6,5,176,5_έκδοση απόφασης έκδοση_απόφασης έκδοση απόφα...,"[έκδοση απόφασης έκδοση, απόφασης έκδοση απόφα...","[Έκδοση Απόφασης, Έκδοση απόφασης, Έκδοση απόφ..."
7,6,143,6_απόρριψη αιτήματος απόρριψη_αιτήματος απόρρι...,"[απόρριψη αιτήματος απόρριψη, αιτήματος απόρρι...","[Απόρριψη αίτησης, Απόρριψη αίτησης, Απόρριψη ..."
8,7,199,7_πρωτοκόλληση αίτησης πρωτοκόλληση_αίτησης πρ...,"[πρωτοκόλληση αίτησης πρωτοκόλληση, αίτησης πρ...","[Πρωτοκόλληση Αίτησης, Πρωτοκόλληση Αίτησης, Π..."
9,8,146,8_παραλαβή δικαιολογητικών παραλαβή_δικαιολογη...,"[παραλαβή δικαιολογητικών παραλαβή, δικαιολογη...","[Παραλαβή αίτησης και δικαιολογητικών, Παραλαβ..."


In [ ]:
freq["Topic"].nunique()

529

We reduced a lot the outliers to 10%, which would be good for our semantic analysis.  Finally have a look if we should merge some topics

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=50)

In [ ]:
topics_to_merge = [[3,12], [2,63], [1,13]] #seem to close
merged_topics = topic_model.merge_topics(docs, topics_to_merge)

In [ ]:
topic_model.update_topics(docs, topics=merged_topics, vectorizer_model=vectorizer_model,  ctfidf_model=ctfidf_model)

In [ ]:
freq = topic_model.get_topic_info(); freq.head(20)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1976,-1_αναφοράς_1ο_σχετικά_1ο μέρος,"[αναφοράς, 1ο, σχετικά, 1ο μέρος, ως, μέτρων, ...","[1ο μέρος: Χρέωση αίτησης σε τμήμα, Υποβολή αί..."
1,0,422,0_εκκίνηση διαδικασίας εκκίνηση_διαδικασίας εκ...,"[εκκίνηση διαδικασίας εκκίνηση, διαδικασίας εκ...","[Εκκίνηση διαδικασίας, Εκκίνηση διαδικασίας, Ε..."
2,1,350,1_συνέχεια_πρωτοβάθμιας_λοιπών δικαιολογητικών...,"[συνέχεια, πρωτοβάθμιας, λοιπών δικαιολογητικώ...",[Παραλαβή και έλεγχος της αίτησης και λοιπών δ...
3,2,343,2_έλεγχος δικαιολογητικών έλεγχος_δικαιολογητι...,"[έλεγχος δικαιολογητικών έλεγχος, δικαιολογητι...","[Έλεγχος δικαιολογητικών, Έλεγχος δικαιολογητι..."
4,3,332,3_δικαιολογητικών ηλεκτρονική υποβολή_ηλεκτρον...,"[δικαιολογητικών ηλεκτρονική υποβολή, ηλεκτρον...",[Ηλεκτρονική Υποβολή Αίτησης και Δικαιολογητικ...
5,4,294,4_γνωμοδότηση_γνωμοδότησης κεντρικού_γνωμοδότη...,"[γνωμοδότηση, γνωμοδότησης κεντρικού, γνωμοδότ...",[Έκδοση γνωμοδότησης του Κεντρικού Αρχαιολογικ...
6,5,206,5_συμβούλιο δευτεροβάθμιας εκπαίδευσης_συμβούλ...,"[συμβούλιο δευτεροβάθμιας εκπαίδευσης, συμβούλ...",[Απόφαση από το Κεντρικό Υπηρεσιακό Συμβούλιο ...
7,6,205,6_παραλαβή δικαιολογητικών πρωτοκόλληση_παραλα...,"[παραλαβή δικαιολογητικών πρωτοκόλληση, παραλα...","[Παραλαβή δικαιολογητικών και πρωτοκόλληση, Πα..."
8,7,199,7_πρωτοκόλληση αίτησης πρωτοκόλληση_αίτησης πρ...,"[πρωτοκόλληση αίτησης πρωτοκόλληση, αίτησης πρ...","[Πρωτοκόλληση αίτησης, Πρωτοκόλληση αίτησης, Π..."
9,8,183,8_υποβολή αίτησης υποβολή_αίτησης υποβολή αίτη...,"[υποβολή αίτησης υποβολή, αίτησης υποβολή αίτη...","[Υποβολή αίτησης, Υποβολή Αίτησης, Υποβολή αίτ..."


And some insights

In [ ]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,Φυσική ταυτοποίηση πολίτη στο ΚΕΠ,17,17_πολίτη φυσική_ταυτοποίηση πολίτη φυσική_πολ...,"[πολίτη φυσική, ταυτοποίηση πολίτη φυσική, πολ...","[Φυσική ταυτοποίηση πολίτη, Φυσική ταυτοποίηση...",πολίτη φυσική - ταυτοποίηση πολίτη φυσική - πο...,0.018152,False
1,Υποβολή αίτησης στο Πληροφοριακό σύστημα,5,5_συμβούλιο δευτεροβάθμιας εκπαίδευσης_συμβούλ...,"[συμβούλιο δευτεροβάθμιας εκπαίδευσης, συμβούλ...",[Απόφαση από το Κεντρικό Υπηρεσιακό Συμβούλιο ...,συμβούλιο δευτεροβάθμιας εκπαίδευσης - συμβούλ...,0.000000,False
2,Εκτύπωση αποδεικτικού φορολογικής ενημερότητας,11,11_φαρμακευτικών προϊόντων_κτηνιατρικών φαρμακ...,"[φαρμακευτικών προϊόντων, κτηνιατρικών φαρμακε...",[Παραλαβή άδειας λιανικής πώλησης κτηνιατρικών...,φαρμακευτικών προϊόντων - κτηνιατρικών φαρμακε...,0.000000,False
3,Εκκίνηση διαδικασίας,0,0_εκκίνηση διαδικασίας εκκίνηση_διαδικασίας εκ...,"[εκκίνηση διαδικασίας εκκίνηση, διαδικασίας εκ...","[Εκκίνηση διαδικασίας, Εκκίνηση διαδικασίας, Ε...",εκκίνηση διαδικασίας εκκίνηση - διαδικασίας εκ...,0.578730,True
4,Παραλαβή αιτήματος για διακίνηση χοιροειδών,127,127_χορήγηση άδειας_παραλαβή αίτησης χορήγηση_...,"[χορήγηση άδειας, παραλαβή αίτησης χορήγηση, α...",[Παραλαβή της αίτησης για χορήγηση άδειας κατη...,χορήγηση άδειας - παραλαβή αίτησης χορήγηση - ...,0.980173,False
...,...,...,...,...,...,...,...,...
19926,Πρωτοκόλληση αίτησης,7,7_πρωτοκόλληση αίτησης πρωτοκόλληση_αίτησης πρ...,"[πρωτοκόλληση αίτησης πρωτοκόλληση, αίτησης πρ...","[Πρωτοκόλληση αίτησης, Πρωτοκόλληση αίτησης, Π...",πρωτοκόλληση αίτησης πρωτοκόλληση - αίτησης πρ...,1.000000,True
19927,Δρομολόγηση αίτησης,179,179_δρομολόγηση αίτησης_αίτησης δρομολόγηση_αί...,"[δρομολόγηση αίτησης, αίτησης δρομολόγηση, αίτ...","[Δρομολόγηση αίτησης, Δρομολόγηση αίτησης, Δρο...",δρομολόγηση αίτησης - αίτησης δρομολόγηση - αί...,0.012452,True
19928,Έλεγχος έννομου συμφέροντος,99,99_έλεγχος έννομου_έλεγχος έννομου συμφέροντος...,"[έλεγχος έννομου, έλεγχος έννομου συμφέροντος,...","[Έλεγχος έννομου συμφέροντος, Έλεγχος έννομου ...",έλεγχος έννομου - έλεγχος έννομου συμφέροντος ...,0.395232,True
19929,Έκδοση πιστοποιητικού,23,23_έκδοση πιστοποιητικού έκδοση_πιστοποιητικού...,"[έκδοση πιστοποιητικού έκδοση, πιστοποιητικού ...","[Έκδοση πιστοποιητικού, Έκδοση πιστοποιητικού,...",έκδοση πιστοποιητικού έκδοση - πιστοποιητικού ...,1.000000,True


In [ ]:
topic_model.get_topic(2)  # Select the most frequent topic

[('έλεγχος δικαιολογητικών έλεγχος', 0.06681469200126203),
 ('δικαιολογητικών έλεγχος δικαιολογητικών', 0.06263541623887003),
 ('έλεγχος δικαιολογητικών', 0.0302321191055562),
 ('λοιπών δικαιολογητικών έλεγχος', 0.009505429050052729),
 ('έλεγχος λοιπών', 0.008242754316648636),
 ('έλεγχος λοιπών δικαιολογητικών', 0.008242754316648636),
 ('δικαιολογητικών έλεγχος λοιπών', 0.008242754316648636),
 ('έλεγχος δικαιολογητικών ελεγχος', 0.008242754316648636),
 ('λοιπών', 0.007967876114403924),
 ('έλεγχος αίτησης δικαιολογητικών', 0.00790309005777129)]

### Attributes

There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |


## Save Topics

In [ ]:
result = topic_model.get_document_info(docs)

In [ ]:
len(result)

19931

In [ ]:
#add process_id for comparison reasons
result = topic_model.get_document_info(docs)

df = df.reset_index(drop=True) #need also to reindex the df

result['process_id'] = df.loc[result.index, 'process_id']

In [ ]:
result.to_csv('topic_full.csv')

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created.

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation.
Instead, we can visualize the topics that were generated in a way very similar to
[LDAvis](https://github.com/cpsievert/LDAvis):

In [ ]:
topic_model.visualize_topics()

Output hidden; open in https://colab.research.google.com to view.

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [ ]:
topic_model.visualize_barchart(top_n_topics=5)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [ ]:
topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

Output hidden; open in https://colab.research.google.com to view.

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [ ]:
topic_model.visualize_term_rank()

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar
to an input search_term. Here, we are going to be searching for topics that closely relate the
search term "vehicle". Then, we extract the most similar topic and check the results:

In [ ]:
similar_topics, similarity = topic_model.find_topics("έλεγχος δικαιολογητικών", top_n=5); similar_topics

[2, 24, 19, 413, 100]

In [ ]:
topic_model.get_topic(2)

[('έλεγχος δικαιολογητικών έλεγχος', 0.06681469200126203),
 ('δικαιολογητικών έλεγχος δικαιολογητικών', 0.06263541623887003),
 ('έλεγχος δικαιολογητικών', 0.0302321191055562),
 ('λοιπών δικαιολογητικών έλεγχος', 0.009505429050052729),
 ('έλεγχος λοιπών', 0.008242754316648636),
 ('έλεγχος λοιπών δικαιολογητικών', 0.008242754316648636),
 ('δικαιολογητικών έλεγχος λοιπών', 0.008242754316648636),
 ('έλεγχος δικαιολογητικών ελεγχος', 0.008242754316648636),
 ('λοιπών', 0.007967876114403924),
 ('έλεγχος αίτησης δικαιολογητικών', 0.00790309005777129)]

## Save the model

In [ ]:
topic_model.save("topic_model_mitos", serialization="pickle")

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [ ]:
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
topic_model.save("/content/gdrive/MyDrive/Colab Notebooks/Topic Modelling", serialization="pytorch", save_ctfidf=True, save_embedding_model=embedding_model)